<h1>Цифровой прорыв — 2021. ФИНАЛ // Кейс Hyper AdTech</h1>
<h2>Решение по определению цифрового портрета аудитории в мобильной среде</h2>
___________________________________________________________________

`Big Data`  `Data Science`

<h3>Команда DST-OFF<span class=\"tocSkip\"></span></h3>

# Постановка задачи
Рекламная платформа Hyper AdTech, специализирующаяся на размещении видеорекламы в мобильных приложениях, поставила задачу создать решение, которое позволило бы максимально точно определить профиль аудитории в  мобильной среде на основе различных косвенных и исторических аудиторных данных.

Для решения кейса вам доступны модельные данные. Данные состоят из:
* Сегментов целевых аудиторий — они параметризуются социально-демографическими атрибутами и/или тематическими интересами Истории активности пользователей в мобильных приложениях — это идентификатор приложения, время и географическая локация 
* Панелистов — каждый панелист принадлежит хотя бы одному сегменту, а действия панелиста размечены в истории активности его сегментами 
* Идентификаторов мобильных устройств, пользователей и панелистов в модельных данных нет. 

Ваша задача — разметить историю активности так, чтобы проставить везде недостающие сегменты.

В рамках кейса  необходимо решить следующие задачи:
1. Обучить классификатор предсказывать сегмент по признакам на тренировочной выборке и проверить работу классификатора на тестовой выборке, в которой отсутствуют метки сегментов;
2. Используя таблицу признаков, кластеризовать пользователей и сравнить найденные кластеры с сегментами, представленными в Dataset.

## Требования к результату
Программное решение, которое
1. на входе получает набор проверочных данных, и
2. выводит таблицу охвата (доля сегмента, или же вероятность получить сегмент в трафике).

Результат по первой задаче:
* Алгоритм, который на входе получает информацию о признаках трафика (категория приложения, время и местоположение пользователя) и на выходе определяет сегменты, к которым принадлежат пользователи
Результат нужно представить в виде таблицы с двумя колонками, где первая колонка - это соответствующий номер записи, а вторая колонка - это предсказанный сегмент.

Результат по второй задаче:
* Алгоритм, который на входе получает аналогичную п.1 информацию о признаках трафика, и выводит:
   * процентное отношение количества каждого определенного вами кластера. Кроме того необходимо обосновать полученный результат, то есть описать признаки трафика, который принадлежит выделенному вами кластеру. По возможности также нужно обосновать результат; если это предполагает выбранный вами алгоритм, то дополнительно выявленными вами признаками принадлежности к кластеру. 
   
Требование к презентации результата:
* Представить процент охвата по сегментам. Для этого нужно посчитать какой процент каждый из типов предсказанных сегментов занимает по отношению ко всей тестовой выборке. 

Поправка:
* Для кластеризации нужен массив размера (n x 5) (с вероятностями классов, n - размер тестовой выборки)
* Таблица по % охватам нужна для задачи кластеризации, её следует привести в презентации вашего решения

# Описание решения

* Изучение данных (этот блокнот)
* Разделение тренировочного и тестового датасетов по дням (блокнот 01-split_csv.ipynb)
* Обучение дневных моделей и предсказание результатов по дням  (блокнот 02-random-forest.ipynb)
* Соединение результатов по дням в общий файл resuls.zip (блокнот 03-result_concate.ipynb)

Ход решения представлен ниже.

In [1]:
#!c1.8
%pip install -U pip icecream seaborn

In [18]:
#!c1.8
import pandas as pd
import numpy as np

import os

import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from icecream import ic

In [3]:
#!c1.8
ic()
df = pd.read_csv('train.csv.zip')
ic()

In [4]:
#!c1.8
ic()
display(df.info(null_counts=True))
ic()

In [5]:
#!c1.8
ic()
display(df.describe(include='all'))
ic()

In [6]:
#!c1.8
ic()
display(df[df.created=='1970-01-01 03:00:00'])
ic()

In [7]:
#!c1.8
ic()
display(df.nunique())
ic()

In [8]:
#!c1.8
ic()
df['date'] = pd.to_datetime(df.created).dt.date
df['month'] = pd.to_datetime(df.created).dt.month
display(df.nunique())
ic()

In [13]:
#!c1.8
display(df['date'].value_counts().sort_index())

In [10]:
#!c1.8
for d in df['date'].unique():
    ic()
    df[df['date']==d].to_csv(f"train_{d}.csv", index=False, compression='zip')
    os.rename(f"train_{d}.csv", f"train_{d}.csv.zip")
ic()

In [14]:
#!c1.8
ic()
df_test = pd.read_csv('test.csv.zip')
ic()

In [15]:
#!c1.8
ic()
df_test['date'] = pd.to_datetime(df_test.created).dt.date
df_test['month'] = pd.to_datetime(df_test.created).dt.month
display(df_test.nunique())
ic()

In [12]:
#!c1.8
ic()
display(df.Segment.value_counts(normalize=True)*100)
ic()

In [16]:
#!c1.8
ic()
display(df.gamecategory.value_counts())
ic()

In [17]:
#!c1.8
ic()
display(df.subgamecategory.value_counts())
ic()

In [25]:
#!c1.8
ic()
display(df.bundle.value_counts()[df.bundle.value_counts()>5000])
ic()

In [111]:
#!c1.8

topgames = df.bundle.value_counts()[:20].index
df_topgames = df[df.bundle.isin(topgames)]
ic()
len(df_topgames)

In [33]:
#!c1.8
ic()
tdf = df_topgames.groupby(by=['bundle', 'date']).date.count().unstack().fillna(0)
ic()
display(tdf/tdf.sum()*100)

In [88]:
#!c1.8
ic()
topsubgamecategory = df.subgamecategory.value_counts()[:25].index
df_topsubgamecategory = df[df.subgamecategory.isin(topsubgamecategory)]
tdf = df_topsubgamecategory.groupby(by=['subgamecategory', 'date']).date.count().unstack().fillna(0)
ic()
tdf['Итого'] = tdf.sum(axis=1)
tdf.sort_values('Итого', ascending=False, inplace=True)
ic()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.1f}'.format
display(tdf/tdf.sum()*100)

In [87]:
#!c1.8
display((tdf/tdf.sum()*100).T.plot(grid=True, figsize=(30,12)).legend(loc='right', bbox_to_anchor=(1, 0.6), ncol=2))

In [89]:
#!c1.8
ic()
topoblast = df.oblast.value_counts()[:25].index
df_topoblast = df[df.oblast.isin(topoblast)]
tdf = df_topoblast.groupby(by=['oblast', 'date']).date.count().unstack().fillna(0)
ic()
tdf['Итого'] = tdf.sum(axis=1)
tdf.sort_values('Итого', ascending=False, inplace=True)
ic()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.1f}'.format
display(tdf/tdf.sum()*100)

In [90]:
#!c1.8
display((tdf/tdf.sum()*100).T.plot(grid=True, figsize=(30,12)).legend(loc='right', bbox_to_anchor=(1, 0.6), ncol=2))

In [175]:
#!c1.8
ic()
topoblast = df.oblast.value_counts()[:25].index
df_topoblast = df[df.oblast.isin(topoblast)]
tdf = df_topoblast.groupby(by=['Segment', 'oblast']).date.count().unstack().fillna(0)
ic()
tdf.sort_values(5, ascending=False, axis=1, inplace=True)
tdf['Итого'] = tdf.sum(axis=1)
ic()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.1f}'.format
display((tdf/tdf.sum()*100).T)

In [176]:
#!c1.8
display((tdf/tdf.sum()*100).T[::-1].plot.barh(stacked=True, figsize=(30,12)).legend(loc='right', bbox_to_anchor=(1, 0.6)))

In [122]:
#!c1.8
ic()
topoblast = df.oblast.value_counts()[:25].index
df_topoblast = df[df.oblast.isin(topoblast)]
tdf = df_topoblast.groupby(by=['month', 'Segment', 'oblast']).date.count().unstack().fillna(0)
ic()
tdf.sort_values((9, 5), ascending=False, axis=1, inplace=True)
tdf['Итого'] = tdf.sum(axis=1)
ic()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.1f}'.format
display((tdf/tdf.sum(level=0)*100).T)

In [ ]:
#!c1.8
ic()
topsubgamecategory = df.subgamecategory.value_counts()[:25].index
df_topsubgamecategory = df[df.subgamecategory.isin(topsubgamecategory)]
tdf = df_topsubgamecategory.groupby(by=['subgamecategory', 'date']).date.count().unstack().fillna(0)
tdf.sort_values((9, 5), ascending=False, axis=1, inplace=True)
tdf['Итого'] = tdf.sum(axis=1)
ic()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.1f}'.format
display((tdf/tdf.sum(level=0)*100).T)

In [124]:
#!c1.8
pd.set_option('display.max_rows', 10)
df[df.created=='1970-01-01 03:00:00']

In [129]:
#!c1.8

# Выделим номер дня недели
df['weekdayid'] = pd.to_datetime(df.date).dt.weekday

# Для наглядности определим текстовую метку дня недели
df['weekday'] = df.weekdayid.map({0: 'Пн', 1: 'Вт', 2: 'Ср', 3: 'Чт', 4: 'Пт', 5: 'Сб', 6: 'Вс'})
ax = df.groupby(by=['date', 'weekday']).date.count().plot(grid=True, kind='bar', figsize=(15, 10), rot=75)
# for x in range(df.date_time.min().weekday()%6,df.date.nunique()+1,7):
#     ax.axvspan(x-1.5, x+0.5, facecolor='grey', alpha=0.2)
plt.show()

In [130]:
#!c1.8
ic()
df_test = pd.read_csv('test.csv.zip')
ic()

In [131]:
#!c1.8
df_test

In [136]:
#!c1.8
ic()
df_test['date'] = pd.to_datetime(df_test.created).dt.date
df_test['month'] = pd.to_datetime(df_test.created).dt.month
ic()
# Выделим номер дня недели
df_test['weekdayid'] = pd.to_datetime(df_test.date).dt.weekday
ic()
# Для наглядности определим текстовую метку дня недели
df_test['weekday'] = df_test.weekdayid.map({0: 'Пн', 1: 'Вт', 2: 'Ср', 3: 'Чт', 4: 'Пт', 5: 'Сб', 6: 'Вс'})
ax = df_test.groupby(by=['date', 'weekday']).date.count().plot(grid=True, kind='bar', figsize=(15, 10), rot=75)
# for x in range(df.date_time.min().weekday()%6,df.date.nunique()+1,7):
#     ax.axvspan(x-1.5, x+0.5, facecolor='grey', alpha=0.2)
plt.show()

In [141]:
#!c1.8
ic()
topoblast = df_test.oblast.value_counts()[:25].index
df_test_topoblast = df_test[df_test.oblast.isin(topoblast)]
tdf = df_test_topoblast.groupby(by=['oblast', 'date']).date.count().unstack().fillna(0)
ic()
tdf['Итого'] = tdf.sum(axis=1)
tdf.sort_values('Итого', ascending=False, inplace=True)
ic()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.1f}'.format
display(tdf/tdf.sum()*100)

In [142]:
#!c1.8
ic()
topgames = df_test.bundle.value_counts()[:20].index
df_topgames = df_test[df_test.bundle.isin(topgames)]
ic()
len(df_topgames)

In [144]:
#!c1.8
ic()
tdf = df_topgames.groupby(by=['bundle', 'date']).date.count().unstack().fillna(0)
ic()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.1f}'.format
display(tdf/tdf.sum()*100)

In [145]:
#!c1.8
ic()
df = df[df.created!='1970-01-01 03:00:00']
df.created = pd.to_datetime(df.created)
df.fillna('', inplace=True)
df.subgamecategory = df.gamecategory + '/' + df.subgamecategory
df.city = df.oblast + '/' + df.city
df.osv = df.os + '/' + df.osv
df.os = df.os.str.lower()
ic()

In [193]:
#!c1.8
ic()
dfr = pd.DataFrame(columns = ['segment', '5min', 'weekdayid', 'month', 'gcid', 'sgcid', 'bid', 'tsid', 'oid', 'cid', 'osid', 'osvid'])

# Сформируем для каждой записи порядковый номер на временнй шкале
# dfr['dtid'] = df.index.map(dict(zip(df.sort_values('created').index, np.arange(len(df)))))
# ic()

# Разобьем время на 15-минутные интервалы (условно-минималная сессия без прерываний видео)
# если в рамках одной сессии показано несколько видео, то фиксируем нескольких пользователей
dfr['5min'] = (df.created.sub(pd.to_datetime('2021-01-01')).dt.total_seconds() // 300).astype(int)
ic()

dfr['weekdayid'] = df.weekdayid
ic()

dfr['month'] = df.month
ic()

dfr['segment'] = df['Segment']
ic()
gamecategory_list = df['gamecategory'].unique()
gamecategory_to_gcid = dict(zip(gamecategory_list, np.arange(len(gamecategory_list))))
dfr['gcid'] = df['gamecategory'].map(gamecategory_to_gcid)
ic()
subgamecategory_list = df['subgamecategory'].sort_values().unique()
subgamecategory_to_sgcid = dict(zip(subgamecategory_list, np.arange(len(subgamecategory_list))))
dfr['sgcid'] = df['subgamecategory'].map(subgamecategory_to_sgcid)
ic()
bundle_list = df['bundle'].unique()
bundle_to_bid = dict(zip(bundle_list, np.arange(len(bundle_list))))
dfr['bid'] = df['bundle'].map(bundle_to_bid)
ic()
shift_list = df['shift'].unique()
shift_to_tsid = dict(zip(shift_list, np.arange(len(shift_list))))
dfr['tsid'] = df['shift'].map(shift_to_tsid)
ic()
oblast_list = df['oblast'].unique()
oblast_to_oid = dict(zip(oblast_list, np.arange(len(oblast_list))))
dfr['oid'] = df['oblast'].map(oblast_to_oid)
ic()
city_list = df['city'].unique()
city_to_cid = dict(zip(city_list, np.arange(len(city_list))))
dfr['cid'] = df['city'].map(city_to_cid)
ic()
os_list = df['os'].unique()
os_to_osid = dict(zip(os_list, np.arange(len(os_list))))
dfr['osid'] = df['os'].map(os_to_osid)
ic()
osv_list = df['osv'].unique()
osv_to_osvid = dict(zip(osv_list, np.arange(len(osv_list))))
dfr['osvid'] = df['osv'].map(osv_to_osvid)
ic()

In [200]:
#!c1.8
dfr

In [194]:
#!c1.8
tdf = dfr.groupby(by=['segment', 'month']).segment.count().unstack().fillna(0)
pd.options.display.float_format = '{:.1f}'.format
(tdf/tdf.sum()*100).T

In [195]:
#!c1.8
pd.options.display.float_format = '{:.1f}'.format
(tdf/tdf.sum()*100)

In [161]:
#!c1.8
pd.set_option('display.max_rows', 10)
display(dfr)

In [177]:
#!c1.8
ic()
my_dpi=96
plt.figure(figsize=(960/my_dpi, 960/my_dpi), dpi=my_dpi)
sns.pairplot(dfr.sample(10000), kind="scatter", hue="segment", plot_kws=dict(s=80, edgecolor="white", linewidth=1, palette='Set2'))
plt.show()
ic()